In [ ]:
import requests
import pandas as pd

# API URL for orders and details
api_url = 'https://www.rohlik.cz/api/v3/orders/delivered?offset=0&limit=1000'
cookies = {
    'cookies1': 'cookies1',
    'cookies2': 'cookies2'
}

In [3]:
# Step 1: Load the list of all orders
orders_response = requests.get(api_url, cookies=cookies)
orders_data = orders_response.json()

# Initialize empty lists to store order and product data, and an empty set to track unique product IDs
dim_order = []
dim_product = []
fact_order_product = []
unique_product_ids = set()  

In [4]:
# Step 2: Loop through each order to load and store order and product details
for order in orders_data:
    order_id = order['id']
    detail_url = f'https://www.rohlik.cz/api/v3/orders/{order_id}'
    
    # Load details of the specific order
    detail_response = requests.get(detail_url, cookies=cookies)
    detail_data = detail_response.json()
    
    # Append basic order information to `dim_order` list
    dim_order.append({
        'order_id': order_id,
        'date': detail_data['orderTime'],
        'items': detail_data['itemsCount'],
        'total_amount': detail_data['priceComposition']['total']['amount'],
        'currency': detail_data['priceComposition']['total']['currency'],
        'delivery_type': detail_data['deliverySlot']['type'],
        'delivery_since': detail_data['deliverySlot']['since'],
        'delivery_till': detail_data['deliverySlot']['till'],
        'payment_type': detail_data['payment'],
        'courier_tip': detail_data['priceComposition'].get('courierTip', {}).get('amount', 0)
    })
    
# Loop through each item in the order to load and store data in `dim_product` (unique IDs) and `fact_order_product` lists
    for item in detail_data['items']:

        # Only add the product to `dim_product` if it hasn't been added before (tracked by `unique_product_ids`)
        if item['id'] not in unique_product_ids:
            unique_product_ids.add(item['id'])  
            dim_product.append({
                'product_id': item['id'],
                'product_name': item['name'],
                'unit': item['unit'],
                'unit_size': item['textualAmount'],
                'price': item['priceComposition']['unit']['amount']
            })


        # Creating a dictionary for fact_order_product to connect orders and products
        order_product = {
            'order_id': order_id,
            'product_id': item['id'],
            'quantity': item['amount'],
            'product_total_price': item['priceComposition']['total']['amount'],
            'courier_tip': detail_data['priceComposition'].get('courierTip', {}).get('amount', 0),
            'compensation': item['compensated']
        }

        # If the item is compensated, include the compensation value
        if item['compensated']:
            order_product['compensation_value'] = item['compensation'].get('compensationValue', None)

        # Append to `fact_order_product` list
        fact_order_product.append(order_product)

In [10]:
# Step 3: Convert lists to DataFrames for further analysis
dim_order_df = pd.DataFrame(dim_order)
dim_product_df = pd.DataFrame(dim_product)
fact_order_product_df = pd.DataFrame(fact_order_product)

fact_order_product_df

,order_id,product_id,quantity,product_total_price,courier_tip,compensation,compensation_value
0,1093203382,709769,1,37.9,50.0,False,NaN
1,1093203382,1294559,1,12.9,50.0,False,NaN
2,1093203382,1296107,1,41.9,50.0,False,NaN
3,1093203382,1316775,1,14.9,50.0,False,NaN
4,1093203382,1317247,1,34.9,50.0,False,NaN
...,...,...,...,...,...,...,...
6910,1004818325,761987,1,25.0,0.0,False,NaN
6911,1004818325,762805,1,18.0,0.0,False,NaN
6912,1004818325,1286927,3,81.0,0.0,False,NaN
6913,1004818325,1293709,6,78.0,0.0,False,NaN


In [11]:
dim_order_df.to_csv('dim_order.csv', index=False, encoding='utf-8-sig')
dim_product_df.to_csv('dim_product.csv', index=False, encoding='utf-8-sig')
fact_order_product_df.to_csv('fact_order_product.csv', index=False, encoding='utf-8-sig')

print("Data successfully split and saved into three files: dim_order.csv, dim_product.csv, and fact_order_product.csv")

Data successfully split and saved into three files: dim_order.csv, dim_product.csv, and fact_order_product.csv
